In [59]:
import requests
#r = requests.post('http://10.30.99.211:8261/gpt_small', data = "What is better for deep learning Python or Matlab?")
#print (r.status_code)

# numpy
import numpy as np
import argparse

# torch
import torch, torch.nn as nn
import torch.nn.functional as F

# pandas
import pandas as pd

# transformers
import pytorch_transformers
from pytorch_transformers import *

# read file
from xml.dom import minidom
import re

#import rank model
from sklearn.ensemble import RandomForestRegressor

# custom extractor
import pickle
from my_functions import extractorRoberta
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
my_extractor = extractorRoberta(my_device = device, model_path = '/notebook/cqas/external_pretrained_models/')
print ("loaded extractors")

from pyterrier import text
from pyterrier.text import scorer

import pyterrier as pt
if not pt.started():
  print ("not")  
  pt.init()

def create_featured_dataset(some_df):
    from pyterrier import text
    from pyterrier.text import scorer
    print (0)
    textscorerTf = text.scorer(body_attr="text", wmodel='BM25', sort=False)
    rtr_bm = textscorerTf.transform(some_df)
    print (1)
    textscorerTf = text.scorer(body_attr="text", wmodel='Tf')
    rtr_tf = textscorerTf.transform(some_df)
    print (2)
    textscorerTf = text.scorer(body_attr="text", wmodel='PL2')
    rtr_pl2 = textscorerTf.transform(some_df)
    print (3)
    textscorerTf = text.scorer(body_attr="text", wmodel='DFIC')
    rtr_dfic = textscorerTf.transform(some_df)
    print (4)
    rtr_pl2_for_merge = rtr_pl2[['qid', 'docno', 'score']]
    rtr_pl2_for_merge = rtr_pl2_for_merge.rename(columns={"score": "score_pl2"})
    print (5)
    rtr_tf_for_merge = rtr_tf[['qid', 'docno', 'score']]
    rtr_tf_for_merge = rtr_tf_for_merge.rename(columns={"score": "score_tf"})
    print (6)
    rtr_bm_for_merge = rtr_bm[['qid', 'docno', 'score']]
    rtr_bm_for_merge = rtr_bm_for_merge.rename(columns={"score": "score_bm"})
    print (7)
    rtr_dfic_for_merge = rtr_dfic[['qid', 'docno', 'score']]
    rtr_dfic_for_merge = rtr_dfic_for_merge.rename(columns={"score": "score_dfic"})
    print (8)
    result = pd.merge(rtr_pl2_for_merge, rtr_tf_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_bm_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_dfic_for_merge, on=["qid", "docno"])
    result = pd.merge(result, some_df, on=["qid", "docno"])
    zipped = [result["score_pl2"], result["score_tf"], result["score_bm"], result["score_dfic"], result['baseline_scores'], result["is_retrieved"], result["ap_score"], result["objs_score"]]
    unzipped_object = zip(*zipped)
    unzipped_list = list(unzipped_object)
    list_of_features = [np.array(elem) for elem in unzipped_list]
    result['features'] = list_of_features
    return result


def extract_objs_asp(model_for_extraction, input_string):
    model_for_extraction.from_string(input_string)
    obj1, obj2, predicates, aspects = model_for_extraction.get_params()
    return (obj1.lower(), obj2.lower(), predicates, aspects)


def count_score1(text, nlu_tuple):
    (obj1, obj2, pred, asp) = nlu_tuple
    r = 1.0
    if (len(obj1) != 0 and len(obj2) != 0):
        if (len(pred) != 0):
            pred = re.sub('[!#?,.:";]', '', pred[0])
            if (obj1 in text and obj2 in text and pred in text):
                r += 1.0
        if (len(asp) != 0):
            asp = re.sub('[!#?,.:";]', '', asp[0])
            if (obj1 in text and obj2 in text and asp in text):
                r += 1.0
        elif (obj1 in text and obj2 in text):
            r = 1.5
        elif (obj1 in text or obj2 in text):
            r = 1.2
    else:
        if (obj1) in text or (obj2) in text:
            r = 1.2
    return r

def count_score_nlu(nlu_tuple):
    if (len(nlu_tuple[0]) == 0):
        return 0.0
    else: return 1.0


def count_score(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and len(obj2) != 0):
        if (obj1 in text):
            r += 1.0
        if (obj2 in text):
            r += 1.0
        for asp in asps:
            if asp in text:
                r += 1.5
        for pred in preds:
            if pred in text:
                r += 1.0
    else:
        if ((obj1) in text and len(obj1)!= 0) or (obj2 in text and len(obj2) != 0):
            r = 1.0
    return r

def count_score_obj(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and obj1 in text):
        r += 1.0
    if (len(obj2) != 0 and obj2 in text):
        r += 1.0
    return r

def count_score_asp_pred(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    o1 = (len(obj1) != 0 and obj1 in text)
    o2 = (len(obj2) != 0 and obj2 in text)
    if (o1 or o2):
        for asp in asps:
            if asp in text:
                r += 0.5
        for pred in preds:
            if pred in text:
                r += 0.5
    return r


def make_scores_obj(query, answers):
    print ("make_scores_obj")
    (obj1, obj2, pred, asp) = extract_objs_asp(extr, query)
    print ("in make scores", obj1, obj2, pred, asp)
    scores_answers = [count_score(cleanhtml(answer), (obj1, obj2, pred, asp)) for answer in answers]
    return scores_answers

def read_xml(filename):
    # convert file filename to list of tuples (number_of_topic, title_of_topic) 
    # input: filename string
    # output: list of corresponding tuples
    answer_list = []
    xmldoc = minidom.parse(filename)
    itemlist = xmldoc.getElementsByTagName('topics')
    print(len(itemlist))
    print(itemlist)
    topic_list = itemlist[0].getElementsByTagName('topic')
    print (len(topic_list))
    for topic in topic_list:
        tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
        answer_list.append(tuple_for_add)
    return answer_list

def make_a_search_request(query):
    # return json
    # json will be processed further
    params = {
            "apikey": "0833a307-97d3-462a-99d9-27db400c70da",
            "query": query,
            "index": ["cw12"],
            "size": 10,
            "pretty": True
        }
    response = requests.get(url = "http://www.chatnoir.eu/api/v1/_search", params = params)
    return response

def clean_punct(s):
    s = re.sub(r'[^\w\s]','',s)
    return s

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()

def add_ranks1(rtr : pd.DataFrame) -> pd.DataFrame:
    """
        Canonical method for adding a rank column which is calculated based on the score attribute
        for each query. Note that the dataframe is NOT sorted by this operation.
        Arguments
            df: dataframe to create rank attribute for
    """
    rtr.drop(columns=["rank"], errors="ignore", inplace=True)
    if len(rtr) == 0:
        rtr["rank"] = pd.Series(index=rtr.index, dtype='int64')
        return rtr
    # -1 assures that first rank will be FIRST_RANK
    rtr["rank"] = rtr.groupby("qid", sort=False)["score"].rank(ascending=False, method="first").astype(int) -1 + 1
    if True:
        rtr.sort_values(["qid", "rank"], ascending=[True,True], inplace=True)
    return rtr


def transform(model, test_DF):
    """
    Predicts the scores for the given topics.

    Args:
        topicsTest(DataFrame): A dataframe with the test topics.
    """
    test_DF = test_DF.copy()

    # check for change in number of features
    found_numf = test_DF.iloc[0].features.shape[0]
    if model.num_f is not None:
        if found_numf != model.num_f:
            raise ValueError("Expected %d features, but found %d features" % (model.num_f, found_numf))
    if hasattr(model.learner, 'feature_importances_'):
        if len(model.learner.feature_importances_) != found_numf:
            raise ValueError("Expected %d features, but found %d features" % (len(model.learner.feature_importances_), found_numf))

    test_DF["score"] = model.learner.predict(np.stack(test_DF["features"].values))
    test_DF["qid"] = [int(elem) for elem in test_DF["qid"]]
    return add_ranks1(test_DF)
    
def write_qrels(output_dir, name, rtr):
    qids = rtr['qid']
    Q0s = [0 for elem in qids]
    docs = rtr['docno']
    scores = rtr['score']
    ranks = rtr['rank']
    tags = [name for elem in qids]
    common_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
    print ("common_list ", common_list[:3])
    with open(output_dir + name +'.qrels', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
    print ("written " + name +'.txt')



/notebook/cqas/external_pretrained_models/roberta.hdf5
/notebook/cqas/external_pretrained_models/vocab_dir
encoder loaded
indexer loaded
cuda device  cuda:0
model path  /notebook/cqas/external_pretrained_models/roberta.hdf5
model loaded
reader loaded
loaded extractors


In [38]:
topics_2020 = read_xml('topics-task-2.xml')
        
topics_2021 = read_xml('topics-task-2-only-titles-2021.xml')
        
with open('list_of_un_answ.pcl', 'rb') as f:
    answers_2020 = pickle.load(f)
            
with open('list_of_un_answ_2021.pcl', 'rb') as f:
    answers_2021 = pickle.load(f)
    
with open('touche2020-task2-relevance-withbaseline.qrels', 'r') as f:
    qrels_lines = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
    qrels = [x.strip().split() for x in qrels_lines] 

qrels_dict = {}
for elem in qrels:
    query, noninf, docno, rank = elem
    if (query in qrels_dict.keys()):
        qrels_dict[query].append((docno, rank))
    else:
        qrels_dict[query] = []
        qrels_dict[query].append((docno, rank))
            
info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"],dtype=object)
info_df_train = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"], dtype=object)
qrels_df = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)
qrels_df_train = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)

1
[<DOM Element: topics at 0x7f3f30e75eb0>]
50
1
[<DOM Element: topics at 0x7f3f30e52550>]
50


In [39]:
for elem in topics_2021:
        qid, query = elem[0], elem[1].strip('\n')
        query = re.sub(r'[^\w\s]','',query)
        query = cleanhtml(query)
        my_extractor.from_string(query)
        structures = my_extractor.get_params()
        
        for ind, answer in enumerate(answers_2021[qid]):
            docno = answer[1]
            score = answer[0]
            text = answer[3]

            #nlu_score = count_score(text, structures)
            objs_score = count_score_obj(text, structures)
            ap_score = count_score_asp_pred(text, structures)
            is_retrieved = count_score_nlu(structures)
            df_row = {"qid":int(qid), "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
            info_df = info_df.append(df_row, ignore_index= True)
            
    #create train df and crels
    
for elem in topics_2020:
        qid, query = elem[0], elem[1].strip('\n')
        query = re.sub(r'[^\w\s]','',query)
        query = cleanhtml(query)
        my_extractor.from_string(query)
        structures = my_extractor.get_params()

        for ind, answer in enumerate(answers_2020[qid]):
            docno = answer[1]
            score = answer[0]
            text = answer[3]

            nlu_score = count_score(text, structures)
            objs_score = count_score_obj(text, structures)
            ap_score = count_score_asp_pred(text, structures)
            is_retrieved = count_score_nlu(structures)
            df_row = {"qid":int(qid), "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
            info_df_train = info_df_train.append(df_row, ignore_index= True)
                
        for qrel in qrels_dict[qid]:
            docno, label = qrel
            df_row = {"qid":int(qid), "docno":docno, "label":label}
            qrels_df_train = qrels_df_train.append(df_row, ignore_index= True)

in extractor get params 0
self prredicates  
extract_objects_predicates tags ['O', 'O', 'B-Predicate', 'O', 'B-Aspect', 'B-Aspect', 'O', 'O', 'B-Object', 'O', 'B-Object']
extract_objects_predicates words ['what', 'is', 'better', 'at', 'reducing', 'fever', 'in', 'children', 'ibuprofen', 'or', 'aspirin']
2
ind, word 2 better
old start, starts 8 set()
string   at reducing fever in children ibuprofen or aspirin
['ibuprofen', 'aspirin']
['better']
['reducing', 'fever']
len(objects) 2
in extractor get params 0
self prredicates  
extract_objects_predicates tags ['O', 'O', 'O', 'O', 'B-Object', 'O']
extract_objects_predicates words ['what', 'are', 'the', 'best', 'rice', 'cookers']
['rice']
[]
[]
len(objects) 1
in extractor get params 0
self prredicates  
extract_objects_predicates tags ['O', 'O', 'O', 'B-Object', 'O', 'B-Object', 'O']
extract_objects_predicates words ['should', 'i', 'buy', 'steel', 'or', 'ceramic', 'knives']
['steel', 'ceramic']
[]
[]
len(objects) 2
in extractor get params 0
s

In [60]:
import pyterrier as pt
if not pt.started():
  print ("not")  
  pt.init()

In [62]:
topics_2020 = read_xml('topics-task-2.xml')
        
topics_2021 = read_xml('topics-task-2-only-titles-2021.xml')
        
with open('list_of_un_answ.pcl', 'rb') as f:
    answers_2020 = pickle.load(f)
            
with open('list_of_un_answ_2021.pcl', 'rb') as f:
    answers_2021 = pickle.load(f)
    
with open('touche2020-task2-relevance-withbaseline.qrels', 'r') as f:
    qrels_lines = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
qrels = [x.strip().split() for x in qrels_lines] 

qrels_dict = {}
for elem in qrels:
    query, noninf, docno, rank = elem
    if (query in qrels_dict.keys()):
        qrels_dict[query].append((docno, rank))
    else:
        qrels_dict[query] = []
        qrels_dict[query].append((docno, rank))
            
            
info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"],dtype=object)
info_df_train = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"], dtype=object)
qrels_df = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)
qrels_df_train = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)

    #create test df
    
for elem in topics_2021:
    qid, query = elem[0], elem[1].strip('\n')
    query = re.sub(r'[^\w\s]','',query)
    query = cleanhtml(query)
    my_extractor.from_string(query)
    structures = my_extractor.get_params()
        
    for ind, answer in enumerate(answers_2021[qid]):
        docno = answer[1]
        score = answer[0]
        text = answer[3]

            #nlu_score = count_score(text, structures)
        objs_score = count_score_obj(text, structures)
        ap_score = count_score_asp_pred(text, structures)
        is_retrieved = count_score_nlu(structures)
        df_row = {"qid":qid, "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
        info_df = info_df.append(df_row, ignore_index= True)
            
    #create train df and crels
    
for elem in topics_2020:
    qid, query = elem[0], elem[1].strip('\n')
    query = re.sub(r'[^\w\s]','',query)
    query = cleanhtml(query)
    my_extractor.from_string(query)
    structures = my_extractor.get_params()

    for ind, answer in enumerate(answers_2020[qid]):
        docno = answer[1]
        score = answer[0]
        text = answer[3]
        label = [el[1] for el in qrels if el[0]==docno]
        nlu_score = count_score(text, structures)
        objs_score = count_score_obj(text, structures)
        ap_score = count_score_asp_pred(text, structures)
        is_retrieved = count_score_nlu(structures)
        df_row = {"qid":qid, "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
        info_df_train = info_df_train.append(df_row, ignore_index= True)
                
    for qrel in qrels_dict[qid]:
        docno, label = qrel
        df_row = {"qid":qid, "docno":docno, "label":label}
        qrels_df_train = qrels_df_train.append(df_row, ignore_index= True)
    
print ("info_df_train.head()", info_df_train.head())
print ("000")
result = create_featured_dataset(info_df_train)
print ("001")
test_ds = create_featured_dataset(info_df)
    
import lightgbm as lgb


4
5
6
7
8


In [70]:
lmart_l = lgb.LGBMRanker(
        task="train",
        silent=False,
        min_data_in_leaf=1,
        min_sum_hessian_in_leaf=1,
        max_bin=255,
        num_leaves=12,
        objective="lambdarank",
        metric="ndcg",
        ndcg_eval_at=[10],
        ndcg_at=[10],
        eval_at=[10],
        learning_rate= .1,
        importance_type="gain",
        num_iterations=100,
        early_stopping_rounds=5
    )
    
val_ds = pd.read_pickle("/notebook/touche2021/test_ds.pkl")
qrels_val_ds = pd.read_pickle("/notebook/touche2021/qrels_df_test.pkl")
print ("result head ", result.head())
rf_lgbm = pt.ltr.apply_learned_model(lmart_l, form="ltr")
rf_lgbm.fit(result, qrels_df_train, topics_and_results_Valid = val_ds, qrelsValid = qrels_val_ds)
print ("fitted")
answs = transform(rf_lgbm, test_ds)

result head    qid                      docno  score_pl2  score_tf   score_bm  score_dfic  \
0   1  clueweb12-0200wb-79-18105  12.977148       9.0  25.306646   23.477962   
1   1  clueweb12-1214wb-88-29751  12.706617      10.0  24.926572   23.321558   
2   1  clueweb12-0706wb-92-20025  11.308204       6.0  23.195164   20.817880   
3   1  clueweb12-1400tw-53-02910   8.896310       4.0  19.173446   17.210010   
4   1  clueweb12-0406wb-58-08558   8.647981       5.0  17.805220   16.049722   

                                         query  \
0  what is the difference between sex and love   
1  what is the difference between sex and love   
2  what is the difference between sex and love   
3  what is the difference between sex and love   
4  what is the difference between sex and love   

                                                text  baseline_scores  \
0  home &gt; articles &gt; sex, sexuality &amp; p...        2270.9827   
1  sex may or may not include penetration. differ...       

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


In [72]:
result.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,features
0,1,clueweb12-0200wb-79-18105,12.977148,9.0,25.306646,23.477962,what is the difference between sex and love,"home &gt; articles &gt; sex, sexuality &amp; p...",2270.9827,0.0,0.0,0.0,"[12.977147771219041, 9.0, 25.306646413477576, ..."
1,1,clueweb12-1214wb-88-29751,12.706617,10.0,24.926572,23.321558,what is the difference between sex and love,sex may or may not include penetration. differ...,2406.7341,0.0,0.0,0.0,"[12.706617142356468, 10.0, 24.926572135926463,..."
2,1,clueweb12-0706wb-92-20025,11.308204,6.0,23.195164,20.817880,what is the difference between sex and love,the prevailing attitude in our society is: let...,1402.0853,0.0,0.0,0.0,"[11.308203688644971, 6.0, 23.19516356116842, 2..."
3,1,clueweb12-1400tw-53-02910,8.896310,4.0,19.173446,17.210010,what is the difference between sex and love,"leave a comment in the past, i’ve had sex, i’v...",1970.1604,0.0,0.0,0.0,"[8.896310063188475, 4.0, 19.173445870962855, 1..."
4,1,clueweb12-0406wb-58-08558,8.647981,5.0,17.805220,16.049722,what is the difference between sex and love,"sex, love and relationship are such a complex ...",1358.9696,0.0,0.0,0.0,"[8.64798145064058, 5.0, 17.80521998357908, 16...."


In [71]:
qrels_df_train.head()

,qid,docno,label
0,1,clueweb12-0001wb-05-12311,0
1,1,clueweb12-1811wb-62-08424,1
2,1,clueweb12-1811wb-62-08423,1
3,1,clueweb12-1217wb-47-14048,0
4,1,clueweb12-1811wb-62-08425,1


In [46]:
from pyterrier import text
from pyterrier.text import scorer
textscorerTf = text.scorer(body_attr="text", wmodel="Tf", sort=False)
rtr_bm = textscorerTf.transform(info_df)

In [47]:
info_df['query'][3]

'what is better at reducing fever in children ibuprofen or aspirin'

In [48]:
info_df['text'][3]

'it is effective in reducing fever and is also prescribed for the treatment of arthritic conditions such rheumatoid- and osteoarthritis, muscle injury, and acute attacks of gout.'

In [49]:
rtr_bm.head()

,qid,docno,rank,score,query
0,51,clueweb12-0001wb-15-05765,NaN,0.0,NaN
1,51,clueweb12-0904wb-53-21907,NaN,0.0,NaN
2,51,clueweb12-1314wb-72-19073,NaN,0.0,NaN
3,51,clueweb12-0311wb-02-29958,NaN,0.0,NaN
4,51,clueweb12-0003wb-21-31533,NaN,0.0,NaN


In [57]:
df = pd.DataFrame(
    [
        [int(0), "what is better at reducing fever in children ibuprofen or aspirin", "d1", "it is effective in reducing fever and is also prescribed for the treatment of arthritic conditions such rheumatoid- and osteoarthritis, muscle injury, and acute attacks of gout"],
        [int(1), "what is better at reducing fever in children ibuprofen or aspirin", "d2", "it is effective in reducing fever and"],
    ], columns=["qid", 'query', 'docno', 'text'], dtype=object)
textscorerTf = pt.text.scorer(body_attr="text", wmodel="Tf")
rtr = textscorerTf.transform(df)

In [58]:
rtr

,qid,docno,rank,score,query
0,0,d1,NaN,0.0,NaN
1,1,d2,NaN,0.0,NaN


In [53]:
info_df.head()

,qid,query,docno,text,baseline_scores,is_retrieved,ap_score,objs_score
0,51,what is better at reducing fever in children i...,clueweb12-0001wb-15-05765,"additionally, these medications can become tox...",1984.9130,1.0,0.0,0.0
1,51,what is better at reducing fever in children i...,clueweb12-0904wb-53-21907,but is your child&#x27;s fever really a cause ...,1700.4545,1.0,0.0,0.0
2,51,what is better at reducing fever in children i...,clueweb12-1314wb-72-19073,"in children, a fever that is equal to or great...",1472.7758,1.0,0.0,0.0
3,51,what is better at reducing fever in children i...,clueweb12-0311wb-02-29958,it is effective in reducing fever and is also ...,1469.3533,1.0,0.0,0.0
4,51,what is better at reducing fever in children i...,clueweb12-0003wb-21-31533,ibuprofen (nurofen and others) is another feve...,1104.5153,1.0,1.0,1.0


In [56]:
type(info_df['qid'][4])

int